In [ ]:
#| default_exp core_v5

Sets the default export module for nbdev. All cells marked `#| export` will be written to `my_blog/core_v5.py`.

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev

In [ ]:
#| export
from fastlite import Database
from pathlib import Path
from datetime import datetime, timedelta
import my_blog.config as config
from urllib.parse import quote, unquote
from fasthtml.common import *
from monsterui.all import *
from fasthtml_auth import AuthManager
from fasthtml.jupyter import *
import re
import frontmatter

#| export
Core imports: `fastlite` for SQLite database, `fasthtml` and `monsterui` for web UI, `fasthtml_auth` for authentication, and `frontmatter` for parsing markdown with YAML metadata.

In [ ]:
#| export
@dataclass
class AppState:
    pdb: Database # for managing posts
    posts_t: Table
    tags_t: Table
    post_tags_t: Table
    auth: AuthManager
    db: Database # For managing users and authorising access

#| export
`AppState` holds all shared application state: the posts database, table references, and auth manager. Passed around instead of using globals.

In [ ]:
#| export
def create_database_tables(pdb: Database):

    class Posts:
        id: int # primary key
        title: str
        slug: str # unique
        content: str
        created: datetime
        updated: datetime
        published: bool
        excerpt: str

    posts = pdb.create(Posts, pk='id', defaults={'published': False}, transform=True)
    posts.create_index(['slug'], unique=True, if_not_exists=True)

    class Tags:
        id: int # primary key
        name: str # unique
    
    tags = pdb.create(Tags, pk='id', transform=True)
    tags.create_index(['name'], unique=True, if_not_exists=True)

    class PostTags:
        post_id: int # foreign key > posts.id
        tag_id: int # foreign key > tags.id
    
    post_tags = pdb.create(PostTags, pk=['post_id', 'tag_id'], transform=True)

    


#| export
Creates the three database tables: `Posts` (blog content), `Tags` (category names), and `PostTags` (many-to-many junction table linking posts to tags).

In [ ]:
#| export
def create_post_database(db_path: str):
    pdb = Database(db_path)
    pdb.execute("PRAGMA foreign_keys = ON")
    create_database_tables(pdb)
    return pdb

#| export
Convenience wrapper that creates a database connection and initializes all tables.

In [ ]:
#| export
def create_app():
    # Create databases and apps, return these within and AppState class.
    # Once created then create the server with srv = serve()
    # Add the routes with rt = app.route
    pdb = create_post_database(config.POSTS_DB_PATH)
    posts_t = pdb.t.posts
    tags_t = pdb.t.tags
    post_tags_t = pdb.t.post_tags
    # Initialize auth database
    auth = AuthManager(
        db_path=str(config.USERS_DB_PATH),
        config={
            'allow_registration': config.ALLOW_REGISTRATION,
            'public_paths': [],  # No public paths - all routes require auth except /auth/*
            'login_path': '/auth/login',
        }
    )
    db = auth.initialize()  
    beforeware = auth.create_beforeware()
    hdrs = (*Theme.blue.headers(highlightjs=True), Script(src="https://unpkg.com/hyperscript.org@0.9.12"),
        Link(rel="icon", type="image/png", href="/static/image/john_pixelated.png"))
    app = FastHTML(
        before=beforeware,
        secret_key=config.SECRET_KEY,
        hdrs=hdrs,
        exts='ws'  # Enable WebSocket support
    )
    config.STATIC_DIR.mkdir(parents=True, exist_ok=True)
    app.mount("/static", StaticFiles(directory=str(config.STATIC_DIR)), name="static")
    auth.register_routes(app, include_admin=True)
    state = AppState(
        pdb=pdb,
        posts_t=posts_t,
        tags_t=tags_t,
        post_tags_t=post_tags_t,
        auth=auth,
        db=db
    )
    return app, state

#| export
Main app factory: initializes both databases, sets up authentication with `fasthtml-auth`, configures headers and static files, and returns the app plus an `AppState` instance.

In [ ]:
#| export
# Route collection for deferred registration
_routes = []

def route(path=None):
    """Decorator to collect routes without registering them immediately.
    Use @route('/path') or @route() for function-name-based paths."""
    def decorator(f):
        _routes.append((path, f))
        return f
    if callable(path):  # @route without parens
        f, path = path, None
        _routes.append((path, f))
        return f
    return decorator

#| export
Route collector decorator. Stores routes in `_routes` list for later registration. Supports both `@route` (path from function name) and `@route('/custom/path')` syntax.

In [ ]:
#| export
def register_routes(app):
    """Register all collected routes with the app."""
    for path, handler in _routes:
        if path:
            app.route(path)(handler)
        else:
            app.route(handler)

#| export
Registers all routes collected by `@route` with the given app. Call this in your `app.py` after importing the module.

In [ ]:
# Initialize and run the app
app, state = create_app()
register_routes(app)  # Register all @route decorated handlers
srv = JupyUvi(app)  # For notebook testing

Initializes and runs the app. `JupyUvi` is used for notebook testing; `rt` is a shorthand for `app.route`.

In [ ]:
#| export
def intro():
    return Article(
        H3("Welcome to my Blog Site", cls="text-2xl font-semibold mb-4"),
        Div(cls="text-base gap-1 text-muted-foreground leading-relaxed space-y-4")(
        P("I created this site to keep a record of things I am interested in.  As such it will largely cover motorhome trips, cycling events and routes that I have done and enjoyed, coding and software development activities I am interested in or engaged with, and technology that I think is worth looking at.  You can find out more about me on my ", hx_link("About", "/about"), " page"),

        P("This site is developed using fastHTML and the Solveit platform, both technologies developed by Jeremy Howard and ",A('Answer.ai', href='https://answer.ai', target="_blank", rel="noopener noreferrer", cls="text-primary underline"), " The desgn is based upon the site of ", A('Jack Hogan.', href='https://jackhogan.net/', target="_blank", rel="noopener noreferrer", cls="text-primary underline"),

        " See my latest blog posts below or find the full list on my ", hx_link("Blog", "/blog"), " page, where posts can be readily filtered by topic.")
        )
    )


#| export
Homepage intro section: returns an `Article` with welcome text and links to About and Blog pages using HTMX-enabled navigation.

In [ ]:
#| export
def hx_attrs(target="#main-content"): return dict(hx_target=target, hx_push_url="true", hx_swap="innerHTML show:window:top")

def hx_link(txt, href, cls="text-primary underline", target="#main-content", **kw):
    return A(txt, href=href, hx_get=href, cls=cls, **hx_attrs(target), **kw)

#| export
`hx_attrs` returns HTMX attributes for partial page updates. `hx_link` creates an anchor that uses both regular `href` and HTMX `hx-get` for SPA-like navigation.

In [ ]:
#| export
def navbar():
    brand = A(Img(src="/static/image/john_pixelated.png", alt="John Richmond", cls="w-6 h-6 rounded-full"), Span("John Richmond "), href="/", hx_get="/", cls="flex items-center gap-2 text-lg font-bold", **hx_attrs())
    links = Div(hx_link("About", "/about"), hx_link("Blog", "/blog"), cls="flex gap-4")
    return Nav(Div(brand, links, cls="flex items-center gap-2 justify-between p-4"), cls="border rounded-lg shadow bg-background")

#| export
Navigation bar with brand logo/name on the left and page links (About, Blog) on the right. Uses flexbox for layout.

In [ ]:
#| export
def x_icon(): return Svg(ft_hx("path", d="M12.6.75h2.454l-5.36 6.142L16 15.25h-4.937l-3.867-5.07-4.425 5.07H.316l5.733-6.57L0 .75h5.063l3.495 4.633L12.601.75Zm-.86 13.028h1.36L4.323 2.145H2.865z"), width=20, height=20, fill="currentColor", viewBox="0 0 16 16", aria_hidden="true")



#| export
Custom X (Twitter) icon as inline SVG—UIkit doesn't include the new X logo.

In [ ]:
#| export
def social_link(icon, href, **kw):
    kw = dict(rel="nofollow noindex") if k == "mail" else dict(target="_blank", rel="noopener noreferrer")
    return A(x_icon() if k == "twitter" else UkIcon(icon, width=20, height=20), href=href, cls="hover:text-primary transition-colors", target="_blank", rel="noopener noreferrer", **kw)

def social_link(k, v):
    ext = dict(rel="nofollow noindex") if k == "mail" else {} if k == "rss" else dict(target="_blank", rel="noopener noreferrer")
    return A(x_icon() if k == "twitter" else UkIcon(k, width=20, height=20), href=v, aria_label=k.title(), cls="hover:text-primary transition-colors", **ext)


def footer():
    links = dict(twitter="https://x.com/@johnWrichmond", youtube="https://youtube.com/@confusedjohn46a", github="https://github.com/fromLittleAcorns")
    icons = Div(*[social_link(k, v) for k, v in links.items()], social_link("mail", "mailto:confusedjohn46@gmail.com"), cls="flex justify-center gap-6 text-muted-foreground")
    return Footer(Divider(), icons, cls="max-w-2xl mx-auto px-6 mt-auto mb-6")

#| export
`social_link` creates social media icon links with appropriate `rel` attributes for security. `footer` assembles the page footer with social icons.

In [ ]:
#| export
def layout(*content, htmx, title=None):
    if htmx and htmx.request: return (Title(title), *content)
    main = Main(*content, cls='w-full max-w-2xl mx-auto px-6 py-8 space-y-8', id="main-content")
    return Title(title), Div(Div(navbar(), cls='max-w-2xl mx-auto px-4 mt-4'), main, footer(), cls="flex flex-col min-h-screen")

#| export
Page layout wrapper. On HTMX requests, returns just the content (for partial swap). On full page loads, wraps content with navbar and footer.

In [ ]:
#| export
def slug_exists(slug):
    if bool(list(state.posts_t.rows_where("slug = ?", [slug], limit=1))):
        return list(state.posts_t.rows_where("slug = ?", [slug], limit=1))[0]['id']
    else:
        return False

#| export
Checks if a slug already exists in the database. Returns the post ID if found, `False` otherwise. Used for update-vs-insert logic.

In [ ]:
#| export
from datetime import datetime

def add_post(title, content, excerpt="", tags=None, published=True):
    slug = title.lower().replace(" ", "-")
    slug = ''.join(c for c in slug if c.isalnum() or c == '-')[:60]
    posts = state.pdb.t.posts
    tags_tbl = state.pdb.t.tags
    post_tags = state.pdb.t.post_tags
    post_id = slug_exists(slug)
    if post_id:
        post = posts.update(dict(id=post_id, title=title, slug=slug, content=content, excerpt=excerpt, 
                                    created=datetime.now(), updated=datetime.now(), published=published))
    else:
        post = posts.insert(dict(title=title, slug=slug, content=content, excerpt=excerpt, 
                                    created=datetime.now(), updated=datetime.now(), published=published))
    post_id = post.id
    if tags:
        for tag in tags:
            # Get existing tags
            existing = list(tags_tbl.rows_where("name = ?", [tag], limit=1))
            # If existing tag then load the relevant id.  If not then create a new one and get the id.
            tag_id = existing[0]['id'] if existing else tags_tbl.insert(dict(name=tag))['id']
            # Check if post_tags exists for this combination and if not add
            existing = list(post_tags.rows_where("post_id= ? and tag_id= ?", [post_id, tag_id], limit=1))
            # print(existing)
            if not existing:
                # Implies no link exists for this post and tag so create one
                post_tags.insert(dict(post_id=post_id, tag_id=tag_id))
    return post_id

In [ ]:
# Example: add a post
add_post(
    title="My First Post",
    content="This is the **content** in markdown.",
    excerpt="A short summary",
    tags=["coding", "technology"]
)

#| export
Creates or updates a post. Generates slug from title, handles tag creation/linking in the junction table. Updates existing posts if slug matches.

In [ ]:
#| export
@route('/blog/{slug}')
def blogpost(htmx, slug: str):
    row = state.posts_t.rows_where("slug = ?", [slug], limit=1)
    p = next((dict(r) for r in row), None)
    if not p: return layout(H2("Not Found"), P("Post not found."), title="Not Found", htmx=htmx)
    p['created'] = datetime.fromisoformat(p['created']) if isinstance(p['created'], str) else p['created']
    content = render_md(p['content'], renderer=EnhancedRenderer)
    image_base = f"/static/image/post_images/{slug}"
    content = process_obsidian_images(content, image_base=image_base)
    content = process_strava_embeddings(content)
    return layout(H1(p['title'], cls="text-3xl font-bold mb-2"), Span(p['created'].strftime('%B %d, %Y'), cls="text-muted-foreground text-sm mb-8 block"), content, 
    Script(src="https://strava-embeds.com/embed.js"), title=p['title'], htmx=htmx)

#| export
Route to display a single blog post. Fetches by slug, parses the datetime, renders markdown content with `render_md`, processes Obsidian images and Strava embeds.

In [ ]:
#| export
@route
def index(htmx):
    posts = get_posts(n=4)
    items = [A(H3(p['title']), P(p['excerpt'], cls="text-muted-foreground"), Span(p['created'].strftime('%d %b %Y'), cls="text-sm text-muted-foreground"), href=f"/blog/{p['slug']}", hx_get=f"/blog/{p['slug']}", cls="block border-b pb-4 hover:bg-muted/50 transition-colors", **hx_attrs()) for p in posts]
    content = Div(*items, cls="space-y-4") if items else P("No posts yet.", cls="text-muted-foreground")
    return layout((intro(), Divider(), Section(H3("Latest Posts", cls="text-xl font-semibold mb-4"), content)), title="Welcome to my Blog", htmx=htmx)

#| export
Homepage route: displays intro section, a divider, and the latest posts as cards.

In [ ]:
#| export
def get_tags(tags_tbl):
    tags = [row.name for row in tags_tbl()]
    return tags


#| export
Returns all tag names from the tags table as a list.

In [ ]:
#| export
def get_post_tags(post_id: int):
    query = """
    SELECT name FROM tags WHERE id IN (SELECT tag_id FROM post_tags WHERE post_id=?)
    """
    tag_name_dicts = state.pdb.q(query, [post_id])
    tag_names = [name['name'] for name in tag_name_dicts]
    return tag_names

#| export
Fetches all tags associated with a specific post via the `post_tags` junction table.

In [ ]:
#| export
def get_posts(n: Union[int, None]=None, tags: Union[List, None] = None):
    if tags:
        place_holders = ','.join('?' * len(tags))
        query = f"""
            SELECT DISTINCT p.* FROM posts p
            JOIN post_tags pt ON p.id = pt.post_id
            JOIN tags t ON pt.tag_id = t.id
            WHERE t.name IN ({place_holders}) AND p.published = True
            ORDER BY p.created DESC
        """
        if n: query += f" LIMIT {n}"
        posts = state.pdb.q(query, tags)
    else:
        posts = list(state.posts_t.rows_where("published = ?", [True], order_by="created DESC", limit=n))
        posts = [dict(r) for r in posts]

    for p in posts:
        p['created'] = datetime.fromisoformat(p['created']) if isinstance(p['created'], str) else p['created']
        p['tags'] = get_post_tags(p['id'])
    return posts

In [ ]:
# Example: get recent posts, optionally filtered by tags
posts = get_posts(n=5)  # Latest 5
posts = get_posts(tags=['cycling', 'motorhome'])  # Filtered by tags

#| export
Main post retrieval function. Optionally filters by tags and limits results. Returns dicts with parsed datetime and tag list attached.

In [ ]:
#| export
def tag_pill(tag_name, selected_tags):
    if tag_name in selected_tags:
        new_tags = selected_tags - {tag_name}
        selected = True
    else:
        new_tags = selected_tags | {tag_name}
        selected = False
    link = f"/blog?tags={','.join(new_tags)}" if new_tags else "/blog"
    cls = [ButtonT.primary if selected else ButtonT.secondary, ButtonT.sm, "rounded-lg"]
    return Button(tag_name, cls=cls, hx_get=link, **hx_attrs("#posts-list"))

#| export
Creates a clickable tag button. Clicking adds/removes the tag from the current filter. Selected tags are styled differently.

In [ ]:
#| export
def tag_filter(selected):
    # Return a div containing all of the tags and their selection status. We also need a button to clear the current selection
    selected: set # a set containing the names of the selected tags
    tags = get_tags(state.tags_t)
    tag_pills = [tag_pill(tag_name, selected) for tag_name in tags]
    clear_btn = Button("X Clear", cls=[ButtonT.default, ButtonT.sm, "rounded-lg"], hx_get="/blog", **hx_attrs("#posts-list"))
    return Div(P("Filter: "), *tag_pills, clear_btn, cls="flex flex-wrap gap-2 items-center", id="tag-filter")

#| export
Builds the tag filter bar: all tag pills plus a "Clear" button to reset filters.

In [ ]:
#| export
def tag_badge(name):
    return Span(name, cls="text-xs px-2 py-1 rounded bg-muted")

#| export
Small styled badge for displaying a tag name on post cards.

In [ ]:
#| export
@route
def blog(htmx, tags:str=None):
    # selected is a SET of the name of the selected tags
    selected = {unquote(t.strip()) for t in (tags or '').split(',') if t.strip()}
    filtered = get_posts(tags=selected)
    tag_filter_div = tag_filter(selected)
    items = [post_card(p) for p in filtered]
    post_content = Div(*items, cls="space-y-2", id="posts-list") if items else P("No posts yet.", cls="text-muted-foreground", id="posts-list")
    if htmx and htmx.target == "posts-list":
        tag_filter_div.attrs['hx-swap-oob'] = 'true'
        return post_content, tag_filter_div
    return layout(H2("Blog"), tag_filter_div, Divider(cls=('my-2')), post_content, title="Blog", htmx=htmx)

#| export
Blog listing route. Parses tag filter from URL, fetches matching posts, renders tag filter + post cards. Uses HTMX OOB swap to update both filter and list on tag clicks.

In [ ]:
#| export
def get_post_image(p):
    # check post content for image (/static/image/post_images/*)
    # If image found then load a thumbnail of it
    img_ptn = r"!\[.*?\]\((/static/image/post_images/[^)]+)\)"
    imgs = re.findall(img_ptn,p["content"])
    img = imgs[0] if len(imgs)>0 else None
    if img:
        img_path = Path(img)
        img_path = config.STATIC_DIR.parent / img_path
        return img_path
    else:
        return None

#| export
Extracts the first image path from a post's markdown content for use as a thumbnail.

In [ ]:
#| export
def post_card(p):
    """Create a card to view a summary of the post including
    - Title (linked)
    - Excerpt
    - possible thumbnail image if one is in the post, to the right of the text
    - Date
    - Tags as small pills?
    - Hover effect?
    """
    img_url = get_post_image(p)
    post = Div(cls="flex gap-2 p-3 -mx-3 rounded-lg hover:bg-muted/50 hover:shadow-lg transition-all cursor-pointer")(
        Div(cls="flex-1")(
            A(cls="flex gap-4 block border-b pb-4 transition-colors")(
                Div(H3(p['title']), P(p['excerpt'], cls="text-muted-foreground"),  
                Div(Span(p['created'].strftime('%d %b %Y'), cls="text-sm text-muted-foreground"), Div([tag_badge(tag) for tag in p["tags"]], cls="flex gap-2 flex-wrap")
                )),
                Img(src=img_url, cls="p-4 max-w-48 h-auto object-contain ml-auto") if img_url else None,
                href=f"/blog/{p['slug']}",
                hx_get=f"/blog/{p['slug']}", **hx_attrs()
            )
        )
    )
    return post

#| export
Renders a post summary card with title, excerpt, date, tags, and optional thumbnail. Entire card is clickable via HTMX.

In [ ]:
#| export
def process_upload(content: bytes, filename: str, slug:str=None):
    # check there is no parent path element and if so just get the filename part
    file_path = Path(filename)
    if file_path.suffix == '.md':
        # post content, extract metadata and save contents to posts table
        md_text = content.decode('utf-8')
        post = frontmatter.loads(md_text)
        # Add checks that the meta data needed is present
        title = post.metadata['title']
        tags = post.metadata['tags']
        excerpt = post.metadata['excerpt']
        slug = title.lower().replace(" ", "-")
        slug = ''.join(c for c in slug if c.isalnum() or c == '-')[:60]
        # Convert obscidian image paths
        content_rewritten = convert_obsidian_images(post.content, f"/static/image/post_images/{slug}")
        # Convert normal markdown image paths
        content_rewritten = rewrite_image_paths( content_rewritten, slug)
        post.content = content_rewritten
        try:
            add_post(title=title, content=post.content, excerpt=excerpt, tags=tags)
        except:
            # add a message that the post could not be added and the failure mode and return to the upload form
            success = False
            message = "Unable to save post"
            return success, message
        return True, "Post saved", slug

    elif file_path.suffix in ['.jpg', '.png', '.jpeg', '.tif', '.svg']:
        # file is an image, save to image folder
        path_to_save = Path(config.POST_IMAGE_DIR) / Path(slug) / file_path.name
        # Create directory if it doen't exist
        path_to_save.parent.mkdir(parents=True, exist_ok=True)
        path_to_save.write_bytes(content)
        return True, f"File {path_to_save.name} saved", slug

    else:
        # unknown file type, raise an error Toast
        return False, f"Unknown file type {filepath.suffix}", slug

#| export
Processes uploaded files. For `.md` files: parses frontmatter, rewrites image paths, saves to database. For images: saves to post-specific subfolder.

In [ ]:
#| export
@route('/admin/upload')
def get(htmx):
    # Create file upload form for the post
    return Div(Div(A('Cancel', href='/', cls=f"{ButtonT.secondary} px-4 py-2"), Upload("Upload Button!", id='upload1', multiple=True), cls='flex gap-2'),
               Div(id='upload-message'),
               UploadZone(DivCentered(Span("Upload Zone"), UkIcon("upload")), id='upload2', accept=['.md', '.jpg', '.jpeg', 'png', 'svg', 'gif'], multiple=True,
               hx_target='#upload-message', hx_trigger='change', hx_post='/admin/upload', hx_swap='innerHTML', hx_include='#upload2', hx_encoding="multipart/form-data"),
               cls='space-y-4')

#| export
Admin upload page (GET): displays a drag-and-drop upload zone for markdown files and images.

In [ ]:
#| export
@route('/admin/upload')    
def post(upload2: list[UploadFile]):
    files = [(f.filename, f.file.read()) for f in upload2]
    md_files = [(n, c) for n, c in files if n.endswith('.md')]
    img_files = [(n, c) for n, c in files if not n.endswith('.md')]
    results = []
    slug = None
    if img_files and len(md_files)==0:
        return Div(Alert("Please upload a post (.md file) with images, or upload images separately", cls=AlertT.warning))
    for name, content in md_files:
        success, message, slug = process_upload(content, name)
        results.append((name, success, message))
    for name, content in img_files:
        success, message, _ = process_upload(content, name, slug=slug)
        results.append((name, success, message))
    header = ["Name", 'Success', 'Message']
    body = [[r[0], r[1], r[2]] for r in results]
    return Div(H2("Post upload results"), TableFromLists(header, body))

#| export
Admin upload handler (POST): processes markdown files first (to get slug), then images. Returns a results table showing success/failure for each file.

In [ ]:
#| export
def rewrite_image_paths(content: str, slug: str) -> str:
    img_ptn = r"(!\[.*?\])\(([^/)]+\.(jpg|jpeg|png|gif|svg))\)"
    replacement = rf"\1(/static/image/post_images/{slug}/\2)"
    return re.sub(img_ptn, replacement, content, flags=re.IGNORECASE)

#| export
Rewrites simple image filenames like `![](image.jpg)` to full paths like `![](/static/image/post_images/{slug}/image.jpg)`. Called at upload time.

In [ ]:
#| export
def convert_obsidian_images(content: str, image_base: str = "/static/image/about") -> str:
    """Convert Obsidian ![[image.ext]] syntax to standard markdown ![](/path/image.ext)"""
    pattern = r'!\[\[([^\]]+\.(jpg|jpeg|png|gif|svg))\]\]'
    replacement = rf'![]({image_base}/\1)'
    return re.sub(pattern, replacement, content, flags=re.IGNORECASE)

def load_md_file(path: str, image_base: str = None) -> str:
    """Load markdown file, optionally converting Obsidian image syntax"""
    content = Path(path).read_text()
    if image_base:
        content = convert_obsidian_images(content, image_base)
    return content

#| export
`convert_obsidian_images` converts Obsidian's `![[image.ext]]` syntax to standard markdown. `load_md_file` loads a markdown file and optionally converts image syntax.

In [ ]:
#| export
def about_content():
    md = load_md_file(config.STATIC_DIR / "content/About.md", image_base="/static/image/about")
    return render_md(md)

#| export
Loads and renders the About page from a markdown file, converting Obsidian image syntax to proper paths.

In [ ]:
#| export
@route
def about(htmx):
    return layout(about_content(),title="About Me", htmx=htmx)

#| export
Route to display the About page.

In [ ]:
#| export
def strava_embed(activity_id: str):
    return Div(cls="strava-embed-placeholder", data_embed_type="activity", data_embed_id=activity_id, data_style="standard")

In [ ]:
# In your markdown, use:
# {{strava:12345678}}
# This will be replaced with an embedded Strava activity

Returns a Strava embed div with the given activity ID. The Strava embed.js script (loaded in headers) will transform this into a full embed.

In [ ]:
#| export
def process_strava_embeddings(page: NotStr):
    page = str(page)
    # Pattern to match {{strava:ID}} possibly wrapped in <p> tags
    pattern_placeholder = r'(<p[^>]*>)?\s*\{\{strava:(\d+)\}\}\s*(</p>)?'
    
    def replace_strava(match):
        activity_id = match.group(2)
        return to_xml(strava_embed(activity_id))
    
    page = re.sub(pattern_placeholder, replace_strava, page)
    return NotStr(page)

Post-processes rendered HTML to find `{{strava:ID}}` placeholders and replace them with actual Strava embed divs.

In [ ]:
#| export
def process_obsidian_images(page: NotStr, image_base: str) -> NotStr:
    page = str(page)
    pattern = new3 = r'''!\[\[(?P<image>[^\]]+\.(jpg|jpeg|png|gif|svg)) # Image part, must be present
(?:\|(?P<size1>\d+))? # First size dimension (with leading pipe symbol (optional)
(?:x(?P<size2>\d+))? # Second size dimension (with leading x)
(?:\|(?P<location>left|right|center))?
\]\] # End of image (must be present)
'''
    
    for match in re.finditer(pattern, page, re.MULTILINE + re.VERBOSE):
        m = match.groupdict()
        
        # 1. Build src path
        src = f"{image_base}/{m['image']}"
        
        # 2. Build style string from size1, size2, location
        styles = []
        if m['size1']:
            styles.append(f"width: {m['size1']}px")
        if m['size2']:
            styles.append(f"height: {m['size2']}px")
        if m['location'] == 'center':
            styles.append("display: block; margin: auto")
        elif m['location'] == 'right':
            styles.append("float: right; margin-left: 1rem")
        elif m['location'] == 'left':
            styles.append("float: left; margin-right: 1rem")
        
        style_str = "; ".join(styles)
        
        # 3. Build img tag and replace
        img_tag = f'<img src="{src}" style="{style_str}">'
        page = page.replace(match.group(0), img_tag)
    
    return NotStr(page)

In [ ]:
# Obsidian image syntax examples:
# ![[image.jpg|300]]           - width 300px
# ![[image.jpg|300|right]]     - float right, text wraps
# ![[image.jpg|300x200|center]] - fixed size, centered

Post-processes rendered HTML to find Obsidian image syntax `![[image.jpg|width|position]]` and replace with styled `<img>` tags. Supports width, height, and positioning (left/right/center).

In [ ]:
#| export
from mistletoe import Document
from monsterui.franken import FrankenRenderer

Additional imports for the custom markdown renderer.

In [ ]:
#| export
class EnhancedRenderer(FrankenRenderer):
    def _is_external(self, url):
        return url.startswith(('http://', 'https://', '//'))

    def render_link(self, token):
        target = self.escape_url(token.target)      
        title = f' title="{self.escape_html(token.title)}"' if token.title else ''
        inner = self.render_inner(token)

        # Determine if we need the new tab attributes
        extra_attrs = ' target="_blank" rel="noopener noreferrer"' if self._is_external(target) else ''

        return f'<a href="{target}"{extra_attrs}{title}>{inner}</a>'

    def render_autolink(self, token):
        target = self.escape_url(token.target)
        inner = self.render_inner(token)
        
        # Autolinks are almost always external, but we'll check anyway
        extra_attrs = ' target="_blank" rel="noopener noreferrer"' if self._is_external(target) else ''
        
        return f'<a href="{target}"{extra_attrs}>{inner}</a>'

Custom markdown renderer that extends `FrankenRenderer`. Automatically adds `target="_blank"` and `rel="noopener noreferrer"` to external links (http/https).

In [ ]:
nbdev.nbdev_export()

Exports all cells marked with `#| export` to the `my_blog/core_v2.py` module file.

I have created a simple python app to use the nbdev module created in by this notebook.  The python file is:
from fasthtml.common import FastHTML, serve
from my_blog.core_v5 import *

# Initialize and run the app
app, state = create_app()
register_routes(app)  # Register all @route decorated handlers
# srv = JupyUvi(app)  # For notebook testing
serve(port=8000)

When I run it the state variable is not present in the module as I have created it in the python app.  What is the best way to solve this without having to pass state into many of the functions, can i make it a global variable in the main app and will it then be visible to the functions

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The issue is that `state` is a module-level variable in your notebook, but when you run the standalone app, the module's `state` is never set—your app creates its own local `state` variable.

The simplest fix is to assign to the module's namespace after `create_app()` returns. In your app file:

```python
from my_blog import core_v5
from my_blog.core_v5 import *

app, state = create_app()
core_v5.state = state  # Set module-level state so route functions can see it
register_routes(app)
serve(port=8000)
```

This works because your route functions look up `state` in the module's global namespace at call time, not at definition time.

An alternative approach would be to add a setter function in your module and export it—but the direct assignment above is the most straightforward for your case.

Does that make sense? Would you like to try it?

I get the following error
Traceback (most recent call last):
  File "/app/data/my-blog/blog.py", line 6, in <module>
    core_v5.state = state
    ^^^^^^^
NameError: name 'core_v5' is not defined
solveit@67d0b8d58de8:~/my-blog$

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You need to import the module itself, not just its contents. Add this import:

```python
from my_blog import core_v5
```

Make sure this line comes *before* the `from my_blog.core_v5 import *` line in your file.